# A. Data import and cleanup

In my experience, around 75% of the time you spend working with data will be fighting to import it and clean it up. For the most part this is just general-purpose programming, but there are a few library routines that will save you from reinventing the wheel.

* [A.0 Reading a simple text file](#readfile)
* [A.1 Reading from a URL](#readurl)
* [A.2 Parsing a log file with regular expressions](#regexp)
* [A.3 Reading JSON from a web service](#json)
* [A.4 Scraping a website with XPath](#xpath)
* [A.5 Reading from an SQL database](#sql)

Treat this section as a collection of recipes and pointers to useful library routines. If you find yourself needing them, you should read the recipe, try it out, then look online for more information about the library functions it suggests.

We'll see plenty of plots, but this notebook is not a tutorial about plotting. For that, have a look at the [matplotlib gallery](https://matplotlib.org/gallery.html), and find recipes for how to achieve the effects you want.

Scientific computing is all about the data: how to handle it, how to plot it, how to reason about it. We could program 
all of our data handling directly in Python or some other low-level language &mdash; but it's better to use high-level expressive libraries, so we spend time thinking about what the data means and not about how it's stored or indexed.

In [ ]:
 For extreme quirks you may need to use the raw Python [`csv.reader`](https://docs.python.org/3/library/csv.html#csv.reader).

## Preamble
At the top of almost every piece of scientific computing work, we'll import these standard modules.

In [2]:
# Import modules, and give them short aliases so we can write e.g. np.foo rather than numpy.foo
import math, random
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import pandas
# The next line is a piece of magic, to let plots appear in our Jupyter notebooks
%matplotlib inline 

## A.0 Reading from a file <a name="readfile"></a>
When your data is a very simple comma-separated value (CSV) file then it's very easy to import. A CSV file looks like this: a header line, then one line per row of the data frame, values separated by commas.
```
"Sepal.Length","Sepal.Width","Petal.Length","Petal.Width","Species"
5.1,3.5,1.4,0.2,"setosa"
4.9,3,1.4,0.2,"setosa"
4.7,3.2,1.3,0.2,"setosa"
4.6,3.1,1.5,0.2,"setosa"
5,3.6,1.4,0.2,"setosa"
```
Here is code to import a simple CSV file.

In [ ]:
with open('data/iris.csv') as f:
    df = pandas.read_csv(f)                   # this returns a pandas.DataFrame
    df = {col:df[col].values for col in df}   # (optional) convert it to dict of vectors

The `with` syntax is useful for files and similar objects which need to be closed/released after we've finished with them, to avoid memory leaks. It's equivalent to calling `f=open('data/iris.csv')`, then running the other commands, then calling `f.close()`.

If your file is nearly a CSV but has some quirks such as comments or a missing header row, experiment with the options in [`pandas.read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) or [`pandas.read_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). For extreme quirks you may need to use the raw Python [`csv.reader`](https://docs.python.org/3/library/csv.html#csv.reader).

## A.1 Reading from a URL<a name="readurl"></a>
You can use the same `pandas.read_csv` function to read any file-like thing, such as a files retrieved over the web.

In [ ]:
import urllib.request  # standard library for web requests
my_url = "https://raw.githubusercontent.com/damonjw/scicomp/master/data/iris.csv"
with urllib.request.urlopen(my_url) as f:
    df = pandas.read_csv(f)
    df = {col:df[col].values for col in df}

You can also read from a string as if it were a file using [`io.StringIO`](https://docs.python.org/3/library/io.html#io.StringIO). This is sometimes handy for debugging. 

## A.2 Parsing a text log file<span id="regexp"></span>
A typical line from a web server log might look like this
```
207.46.13.169 - - [27/Aug/2017:06:52:11 +0000] "GET /marcus/essay/st&h2.html HTTP/1.1" 200 3881 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"
```
where (according to the [Apache web server documentation](https://httpd.apache.org/docs/2.4/logs.html#combined)) the pieces are

* **`207.46.13.169`** 
The IP address that made the request
* **`-`** 
The identity of the client; `-` means not available
* **`-`**
The userid of the logged-in user who made the request; `-` means not available
* **`[27/Aug/2017:06:52:11 +0000]`**
The time the request was received
* **`"GET /marcus/essay/st&h2.html HTTP/1.1"`**
The type of request, what was requested, and the connection type
* **`200`**
The [http status code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) (200 means OK)
* **`3881`**
The size of the object returned to the client, in bytes
* **`-`**
The referrer URL; `-` means not available
* **`"Mozilla/5.0 (...)"`**
The browser type. The substring `bingbot` here tells us that the request comes from Microsoft Bing's web crawler.

To extract these pieces from a single line of the log file, the best tool is [regular expressions](https://docs.python.org/3.4/library/re.html), a mini-language for string matching that is common across many programming languages. The syntax is terse and takes a lot of practice. I like to start with a small string pattern and incrementally build it up, testing as I go.

In [ ]:
import re    # standard Python module for regular expressions
s = """
207.46.13.169 - - [27/Aug/2017:06:52:11 +0000] "GET /marcus/essay/st&h2.html HTTP/1.1" 
200 3881 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X)
AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53
(compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"
"""

# First attempt: match the first two items in the log line.
# If my pattern is right, re.match returns an object.
# If my pattern is wrong, re.match returns None.
pattern_test = r'\s*(\S+)\s*(\S+)'
re.match(pattern_test, s)

In [ ]:
# This is the full pattern I built up to. Python lets us add verbose comments
# to the pattern, which is handy for remembering what your code does when you look
# at it the next morning.
pattern = r'''(?x)  #   flag saying that this pattern has comments
\s*                 #   any whitespace at the start of the string
(?P<ip>\S+)         # one or more non-space characters: the IP address
\s+                 #   one or more spaces
(?P<client>\S+)     # the client identity
\s+
(?P<user>\S+)       # the userid
\s+
\[(?P<t>[^\]]*)\]   # [, then any number of not-] characters, then ]: the timestamp
\s+
"(?P<req>[^"]*)"    # ", then any number of not-" characters, then ": the request string
\s+
(?P<status>\d+)     # one or more numerical digits: the http status code
\s+
(?P<size>\d+)       # one or more numerical digits: the size
\s+
"(?P<ref>[^"]*)"    # the referrer URL
\s+
"(?P<ua>[^"]*)"     # the user agent i.e. browser type
'''
m = re.match(pattern, s)
m.groupdict()       # returns a dictionary of all the named sub-patterns

**Exercise.** Why not use `s.split(' ')`?

How do we extract the fields from a full log file? The standard Python code is
```
with open(myfile) as f:
    for line in f:
        m = re.match(pattern, line)
        # store the fields from m.groups() or m.groupdict() somewhere appropriate
```
Alternatively, `numpy` has a handy shortcut for reading in an entire file and splitting it via a regular expression:

In [ ]:
# Split the file into an array, one row per line, one column per field in the pattern
df = np.fromregex('data/access_short.log', pattern, dtype=np.unicode_)

# Make a dictionary out of the columns, according to the named fields in the pattern
df = {k: df[:,v-1] for k,v in re.compile(pattern).groupindex.items()}

## A.3 Importing from a web data service<a name="json"></a>
More and more forward-thinking companies and government services make data available by simple web requests. Here is an example, importing river levels from the UK's [real-time flood monitoring API](http://environment.data.gov.uk/flood-monitoring/doc/reference).

The first step is to import the Python module for making web requests. When I'm developing data code I like to build it up in small steps, which means lots of repeated requests, so I also like to use another Python module which caches responses. This means I don't hammer the service unnecessarily.

In [ ]:
import requests, requests_cache
requests_cache.install_cache('floodsystem', backend='memory')

The [flood monitoring API documentation](http://environment.data.gov.uk/flood-monitoring/doc/reference) tells us the URL for fetching a list of all stations, `{root}/id/stations`. Let's try it.

In [ ]:
# Make the request. This should print out <Response [200]>, meaning successfully retrieved
stations_resp = requests.get('http://environment.data.gov.uk/flood-monitoring/id/stations')
stations_resp

Now let's look at the body of the response. It's likely to be very long, so we'll only print out the first 300 characters.

In [ ]:
stations_resp.text[:300]

It looks like [JSON](https://en.wikipedia.org/wiki/JSON), "JavaScript Object Notation", a common format for web data services. It's easy to turn it it into Python dictionaries and lists, and to explore what it contains. (Alternatively, just read the web service documentation, if you trust it!)

In [ ]:
x = stations_resp.json()
print(x.__class__)                        # The top-level response is a dictionary
print(x.keys())                           # What are its keys? 'items' sounds promising.
print(x['items'].__class__)               # 'items' is a list,
print(len(x['items']))                    # with 4363 elements.

stations = stations_resp.json()['items']  # This is what we're really after
stations[0]                               # Look at a single sensor

Data from web services is often patchy and inconsistent, so your code for processing it should be full of error handling. For example,
```
rivers = [s['riverName'] for s in stations]
```
will fail because `riverName` isn't present for all stations. You might use one of these instead:
```
rivers = [s.get('riverName', None) for s in stations]
rivers = [s['riverName'] for s in stations if 'riverName' in s]
```
Here's how we might turn the data into a data frame:

In [ ]:
df = {
    'label': np.array([s['label'] for s in stations]),
    'riverName': np.array([s.get('riverName',None) for s in stations]),
    'town': np.array([s.get('town',None) for s in stations])
}

# Each station has zero or more measures. Pick out the first 'water level' measure at each.
# The field measure['@id'] is the url to use to get water level readings.
measures = [[m for m in s.get('measures',{}) if m['parameter']=='level'] for s in stations]
df['url'] = np.array([ms[0]['@id'] if len(ms)>0 else None for ms in measures])

## A.4 Scraping a website<a name="xpath"></a>
There are fascinating stories to be discovered from public data, and sometimes you have to work to scrape it from web pages. Here's [an acount](https://onlinejournalismblog.com/2016/11/29/how-the-bbc-england-data-unit-scraped-airport-noise-complaints/) by a BBC data journalist. We'll work with a very simple example: extracting results of the Oxford / Cambridge boat race from the [Wikipedia table](https://en.wikipedia.org/wiki/List_of_The_Boat_Race_results#Main_race).

I recommend using [XPath queries](https://www.w3.org/TR/xpath20/) from the [`lxml`](http://lxml.de/) module.
XPath is a powerful mini-language for extracting data from hierarchical documents, with wide support across many programming languages &mdash; think of it as regular expressions but for html rather than plain text. If you want to scrape websites then it's worth finding a tutorial and learning XPath. For this course, we'll just see how to use XPath in Python.

The first step is to install `lxml`, which is not included with Python.

In [ ]:
!pip3 install lxml

Now we'll fetch the web page and parse the contents. Most web pages are badly-formatted html (sections not properly closed, etc.), and `lxml.html.fromstring` makes a reasonable attempt to make sense of it.

In [ ]:
import requests
import lxml.html
boatrace_url = 'https://en.wikipedia.org/wiki/List_of_The_Boat_Race_results'
resp = requests.get(boatrace_url)
doc = lxml.html.fromstring(resp.content)

This gives us `doc`, the root `<html>` element, which we can inspect.

In [ ]:
print(doc.tag)               # the type of element
print(len(doc))              # the number of children
print([n.tag for n in doc])  # tags of its children, <head> and <body>
print(doc.attrib)            # get the attributes, e.g. <html class="client-nojs" lang="en" dir="ltr">
print(doc.text, doc.tail)    # any text directly under in this element

We want to pull out a particular element from the document, namely the table with boat race results, so we need to work out how to refer to it in XPath. The Chrome webbrowser has a handy tool to help with this. Go to the page you're interested in, and click on View | Developer | Developer Tools. Click on the element-selector button at the top left:
<img src="fig/xpath1.png" alt="use 'select element' mode">
Go back to the web page, and click on a piece close to what you want to select. I clicked on the top left cell of the table:
<img src="fig/xpath2.png" alt="click roughly where you want" style="height:8em">
Go back to the developer tools window, and navigate to the exact element you want. Here, we want the table. Right-click and choose Copy | Copy XPath. 
<img src="fig/xpath3.png" alt="copy the XPath of the element you want" style="height:8em">
It gave me the XPath location `"//*[@id="mw-content-text"]/div/table[2]"`. Now we can extract the data.

In [ ]:
# Pick out the table element.
# (XPath queries return lists, but I only want one item, hence the [0].)
table = doc.xpath('//*[@id="mw-content-text"]/div/table[2]')[0]

# Get a list of all rows i.e. <tr> elements inside the table.
# Print one, to check things look OK.
rows = table.xpath('.//tr')
print(lxml.etree.tostring(rows[1], encoding='unicode'))

# Extract the timestamp and winner columns.
# The timestamp is in the second child, in a <span> element with class "sortkey".
# The winner is in the third child.
df = {'t': [row[1].xpath('.//span[contains(@class, "sortkey")]')[0].text for row in rows[1:]],
      'winner': [row[2].text for row in rows[1:]]}

You should consider the ethics of your web scraping. Here are some thoughts:
from [Sophie Chou at the MIT Media Lab](http://www.storybench.org/to-scrape-or-not-to-scrape-the-technical-and-ethical-challenges-of-collecting-data-off-the-web/), and the [data journalist N&auml;el Shiab](https://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/).

## A.5 Reading from an SQL database

In [3]:
import psycopg2
import json
import gmplot
import tempfile
import webbrowser

with open('/Users/djw/djw1005/etc/credentials.json') as f:
    creds = json.load(f)['db']['local']
conn = psycopg2.connect(user=creds['user'], dbname=creds['dbname'])

In [32]:
cmd = '''
SELECT *
FROM flood_stations AS s JOIN flood_measures AS m ON (m.station_uri = s.uri) 
WHERE river = 'River Cam' OR town = 'Cambridge'
'''
df = pandas.read_sql(cmd, conn)
df

,index,uri,catchment,river,town,lat,lng,index,measure_id,uri,station_uri,qualifier,parameter,period,unit,valuetype,low,high
0,345,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),River Cam,Great Chesterford,52.061730,0.194279,397,398,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,m,instantaneous,0.109,0.333
1,800,http://environment.data.gov.uk/flood-monitorin...,"Parrett, Brue and West Somerset Streams",River Cam,Weston Bampfylde,51.023159,-2.565568,918,919,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,m,instantaneous,0.026,0.600
2,1272,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),River Cam,Milton,52.236542,0.176925,1454,1455,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.218,0.294
3,1435,http://environment.data.gov.uk/flood-monitorin...,Severn Vale,River Cam,Cam,51.699985,-2.360238,1635,1636,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.578,0.782
4,1492,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),River Cam,Cambridge,52.212835,0.120872,1701,1702,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.637,0.740
5,1509,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),River Cam,Great Shelford,52.134462,0.141784,1723,1724,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,m,instantaneous,0.127,0.395
6,1568,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),Bin Brook,Cambridge,52.197227,0.087527,1791,1792,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.057,0.368
7,1635,http://environment.data.gov.uk/flood-monitorin...,Severn Vale,River Cam,Cambridge,51.730432,-2.362218,1864,1865,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.141,1.250
8,1668,http://environment.data.gov.uk/flood-monitorin...,Cam and Ely Ouse (Including South Level),River Cam,Waterbeach,52.268775,0.208550,1901,1902,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Stage,Water Level,900.0,mASD,instantaneous,0.240,0.316


In [33]:
gmap = gmplot.GoogleMapPlotter(52.212, 0.1208, 16)
gmap.scatter(df['lat'], df['lng'], 'red', size=500, marker=False)

with tempfile.NamedTemporaryFile(delete=False, suffix='.html') as f:
    gmap.draw(f.name)
    webbrowser.open('file://' + f.name)

In [41]:
cmd = '''
SELECT s.label, s.id, lat, lng, town, river, measure_id, qualifier, parameter, period, low, high, valuetype, t, value
FROM flood_stations AS s
    JOIN flood_measures AS m ON (m.station_uri = s.uri)
    JOIN flood_readings AS r USING (measure_id)
WHERE s.river = 'River Cam' or s.town = 'Cambridge'
'''
df = pandas.read_sql(cmd, conn)

In [42]:
df['value'] = df['value'].values.astype('float')

In [43]:
df.to_csv('data/flood.csv', index=False)

## A.6 Wrangling with time